In [276]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import matplotlib.patches as mp
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


In [161]:
IBGEFILE_UNITS = pd.read_csv("data\IBGEFILE.UNITS.csv")

In [162]:
BGEIDSC_EF2EFFP = pd.read_csv("data\BGEIDSC.EF2EFFP.csv")

In [163]:
BGETCHDATA_PTCHTRANH = pd.read_csv("data\BGETCHDATA.PTCHTRANH.csv")

In [164]:
Extranet2_ExactFuelEvents = pd.read_csv("data\Extranet2.ExactFuelEvents.csv")

In [166]:
Extranet2_ExactFuelTankLevels = pd.read_csv("data\Extranet2.ExactFuelTankLevels.csv")

In [265]:
df_levels = Extranet2_ExactFuelTankLevels[['TankId', 'TankLevelPercent', 'TankLevelGallons', 'ExactFuelEventId']]

In [267]:
df_events = Extranet2_ExactFuelEvents[['Id', 'EventTimeStamp', 'EquipmentID','DriverID', 'Latitude', 'Longitude','Speed',
                          'Heading', 'Odometer', 'IgnitionStatus','EFReportReason']]

In [268]:
df_optimization = BGEIDSC_EF2EFFP[['TRUCK', 'TANKCAP']]

In [270]:
df_transaction = BGETCHDATA_PTCHTRANH[['TRNID', 'TRNDAT', 'TRNTIM', 'TRNTS#','TRNDRI', 'TRNUNT', 'TRNPI1', 'TRNPQ1']]

In [279]:
df_transaction = df_transaction[~df_transaction['TRNPI1'].isin(['SCLE', 'CADV', 'DEFD', 'ADD ', 'OIL ', 'UREA', 'DEFC'])] 

In [280]:
df_events_30=df_events[df_events['Speed'] <= 30]

In [281]:
df_event_level = pd.merge(df_events, df_levels, how='inner', left_on='Id', right_on = 'ExactFuelEventId')

## Cast keys to common data type 

In [277]:
df_transaction['TRNUNT'] = df_transaction['TRNUNT'].astype(str);
df_optimization['TRUCK'] = df_optimization['TRUCK'].astype(str);
#IBGEFILE_UNITS['UNUNIT'] = IBGEFILE_UNITS['UNUNIT'].astype(str);
df_events['Id'] = df_events['Id'].astype(str);
df_levels['ExactFuelEventId'] = df_levels['ExactFuelEventId'].astype(str);
df_events['EquipmentID'] = df_events['EquipmentID'].astype(str);

## Clean column values


In [278]:
df_transaction['TRNUNT'] = df_transaction['TRNUNT'].apply(lambda x: x.strip()).replace('', np.nan);
df_optimization['TRUCK'] = df_optimization['TRUCK'].apply(lambda x: x.strip()).replace('', np.nan);
#IBGEFILE_UNITS['UNUNIT'] = IBGEFILE_UNITS['UNUNIT'].apply(lambda x: x.strip()).replace('', np.nan);
df_events['Id'] = df_events['Id'].apply(lambda x: x.strip()).replace('', np.nan);
df_levels['ExactFuelEventId'] = df_levels['ExactFuelEventId'].apply(lambda x: x.strip()).replace('', np.nan);
df_events['EquipmentID'] = df_events['EquipmentID'].apply(lambda x: x.strip()).replace('', np.nan);

In [238]:
BGETCHDATA_PTCHTRANH['TRNUNT'] = BGETCHDATA_PTCHTRANH['TRNUNT'].astype('category')
BGEIDSC_EF2EFFP['TRUCK'] = BGEIDSC_EF2EFFP['TRUCK'].astype('category')
IBGEFILE_UNITS['UNUNIT'] = IBGEFILE_UNITS['UNUNIT'].astype('category')
Extranet2_ExactFuelEvents['Id'] = Extranet2_ExactFuelEvents['Id'].astype('category')
Extranet2_ExactFuelTankLevels['ExactFuelEventId'] = Extranet2_ExactFuelTankLevels['ExactFuelEventId'].astype('category')
Extranet2_ExactFuelEvents['EquipmentID'] = Extranet2_ExactFuelEvents['EquipmentID'].astype('category')

## Merge Dfs together

In [239]:
A = pd.merge(BGETCHDATA_PTCHTRANH, IBGEFILE_UNITS, how='inner', left_on='TRNUNT', right_on = 'UNUNIT')
A

,TRNID,TRNDAT,TRNTIM,TRNTS#,TRNTSN,TRNSTC,TRNST,TRNDRI,TRNUNT,TRNPI1,...,TRNPQ3,TRNPI4,TRNPQ4,TRNPI5,TRNPQ5,TRNPI6,TRNPQ6,UNUNIT,UNYEAR,UNMAKE
0,42574765,20170101,20,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,...,0.0,,0.0,,0.0,,0,1564,2013,VOLVO
1,42614106,20170101,1712,516130,PILOT NITRO 243,NITRO,WV,ULAL,1564,ULSD,...,0.0,,0.0,,0.0,,0,1564,2013,VOLVO
2,42672413,20170102,1529,524100,PFJ SOUTHEAST 7976,TROUTMAN,NC,ULAL,1564,DEFD,...,0.0,,0.0,,0.0,,0,1564,2013,VOLVO
3,42873010,20170104,1152,516012,PILOT LAVERGNE 052,LAVERGNE,TN,ULAL,1564,ULSD,...,0.0,,0.0,,0.0,,0,1564,2013,VOLVO
4,42882523,20170104,1300,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,ULAL,1564,ULSD,...,0.0,,0.0,,0.0,,0,1564,2013,VOLVO
5,43280882,20170107,1520,231064,FJ-BEAVER DAM 695,BEAVER DAM,OH,ULAL,1564,ULSD,...,0.0,,0.0,,0.0,,0,1564,2013,VOLVO
6,43394310,20170109,203,231060,FJ-CARLISLE 708,CARLISLE,PA,ULAL,1564,SCLE,...,0.0,,0.0,,0.0,,0,1564,2013,VOLVO
7,43596583,20170110,1509,231846,FJ-MAX MEADOWS 750,FORT CHISWELL,VA,ULAL,1564,ULSD,...,1.0,,0.0,,0.0,,0,1564,2013,VOLVO
8,43719948,20170111,1339,521442,TIMEOUT MARKET #22 & TC,NEWPORT,TN,ULAL,1564,SCLE,...,0.0,,0.0,,0.0,,0,1564,2013,VOLVO
9,44020016,20170113,1645,516264,PILOT MURFREESBORO 404,MURFREESBORO,TN,ULAL,1564,ULSD,...,0.0,,0.0,,0.0,,0,1564,2013,VOLVO


In [240]:
B = pd.merge(A, BGEIDSC_EF2EFFP, how = 'inner', left_on = 'UNUNIT', right_on = 'TRUCK')
B

,TRNID,TRNDAT,TRNTIM,TRNTS#,TRNTSN,TRNSTC,TRNST,TRNDRI,TRNUNT,TRNPI1,...,UNUNIT,UNYEAR,UNMAKE,TRUCK,ACTIVE,SENDFUEL,SENDROUTE,TANKCAP,AVGMPG,SATTYPE
0,42574765,20170101,20,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,...,1564,2013,VOLVO,1564,Y,Y,Y,250,6.0,
1,42614106,20170101,1712,516130,PILOT NITRO 243,NITRO,WV,ULAL,1564,ULSD,...,1564,2013,VOLVO,1564,Y,Y,Y,250,6.0,
2,42672413,20170102,1529,524100,PFJ SOUTHEAST 7976,TROUTMAN,NC,ULAL,1564,DEFD,...,1564,2013,VOLVO,1564,Y,Y,Y,250,6.0,
3,42873010,20170104,1152,516012,PILOT LAVERGNE 052,LAVERGNE,TN,ULAL,1564,ULSD,...,1564,2013,VOLVO,1564,Y,Y,Y,250,6.0,
4,42882523,20170104,1300,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,ULAL,1564,ULSD,...,1564,2013,VOLVO,1564,Y,Y,Y,250,6.0,
5,43280882,20170107,1520,231064,FJ-BEAVER DAM 695,BEAVER DAM,OH,ULAL,1564,ULSD,...,1564,2013,VOLVO,1564,Y,Y,Y,250,6.0,
6,43394310,20170109,203,231060,FJ-CARLISLE 708,CARLISLE,PA,ULAL,1564,SCLE,...,1564,2013,VOLVO,1564,Y,Y,Y,250,6.0,
7,43596583,20170110,1509,231846,FJ-MAX MEADOWS 750,FORT CHISWELL,VA,ULAL,1564,ULSD,...,1564,2013,VOLVO,1564,Y,Y,Y,250,6.0,
8,43719948,20170111,1339,521442,TIMEOUT MARKET #22 & TC,NEWPORT,TN,ULAL,1564,SCLE,...,1564,2013,VOLVO,1564,Y,Y,Y,250,6.0,
9,44020016,20170113,1645,516264,PILOT MURFREESBORO 404,MURFREESBORO,TN,ULAL,1564,ULSD,...,1564,2013,VOLVO,1564,Y,Y,Y,250,6.0,


In [242]:
C = pd.merge(Extranet2_ExactFuelEvents, Extranet2_ExactFuelTankLevels, how='inner', left_on='Id', right_on = 'ExactFuelEventId')
C

,Id_x,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason,Id_y,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
0,12649083,13640241,2017-01-01 00:00:32.387,1992,105369677,tractor,BATK,30.34642,-81.70724,2017-01-01 00:00:30.000,0,0.0,39895.5,2,1,10375449,1,50.7,101.40,12649083
1,12649084,13640242,2017-01-01 00:01:24.867,1712,105437938,tractor,TAYANT,36.93953,-84.09541,2017-01-01 00:01:27.000,0,0.0,360378.8,1,2,10375450,1,78.0,156.00,12649084
2,12649092,13640251,2017-01-01 00:04:15.053,1713,105438293,tractor,RUSR,35.58664,-84.52786,2017-01-01 00:04:17.000,67,219.5,377577.4,1,2,10375458,1,31.7,63.40,12649092
3,12649102,13640261,2017-01-01 00:04:10.260,1585,105301976,tractor,MCAD,35.15430,-90.14263,2017-01-01 00:04:10.000,0,0.0,513931.8,1,2,10375468,1,43.2,108.00,12649102
4,12649103,13640272,2017-01-01 00:10:13.187,1564,105351785,tractor,ULAL,39.90268,-84.19254,2017-01-01 00:10:11.000,3,91.0,498691.0,1,2,10375469,1,50.3,125.75,12649103
5,12649104,13640273,2017-01-01 00:11:08.123,1711,105321298,tractor,FOSJA,30.51558,-81.63283,2017-01-01 00:11:10.000,3,127.6,342948.0,1,2,10375470,1,81.7,163.40,12649104
6,12649105,13640274,2017-01-01 00:11:48.803,1693,105356123,tractor,THOCHA,36.18582,-86.26805,2017-01-01 00:11:50.000,0,333.2,340043.7,1,2,10375471,1,37.4,74.80,12649105
7,12649106,13640275,2017-01-01 00:12:26.077,1787,105455670,tractor,RODMAR,35.16910,-80.88830,2017-01-01 00:12:27.000,53,193.1,224306.6,1,2,10375472,1,62.5,125.00,12649106
8,12649107,13640276,2017-01-01 00:13:02.450,1992,105369677,tractor,BATK,30.34646,-81.70727,2017-01-01 00:13:00.000,0,0.0,39895.5,1,0,10375473,1,52.0,104.00,12649107
9,12649108,13640277,2017-01-01 00:13:38.137,1711,105321298,tractor,FOSJA,30.51642,-81.63271,2017-01-01 00:13:40.000,0,96.3,342948.1,2,1,10375474,1,79.8,159.60,12649108


In [244]:
#Fuel_Transactions = pd.merge(C, B, how='inner', left_on='EquipmentID', right_on = 'TRUCK')
#Fuel_Transactions

In [64]:
Fuel_Transactions.head(100)
#Fuel_Transactions.shape
#Note: It's reccomended that TankLevelPercent not dip below 50%

,Id_x,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason,Id_y,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
0,12649083,13640241,2017-01-01 00:00:32.387,1992,105369677,tractor,BATK,30.34642,-81.70724,2017-01-01 00:00:30.000,0,0.0,39895.5,2,1,10375449,1,50.7,101.40,12649083
1,12649084,13640242,2017-01-01 00:01:24.867,1712,105437938,tractor,TAYANT,36.93953,-84.09541,2017-01-01 00:01:27.000,0,0.0,360378.8,1,2,10375450,1,78.0,156.00,12649084
2,12649092,13640251,2017-01-01 00:04:15.053,1713,105438293,tractor,RUSR,35.58664,-84.52786,2017-01-01 00:04:17.000,67,219.5,377577.4,1,2,10375458,1,31.7,63.40,12649092
3,12649102,13640261,2017-01-01 00:04:10.260,1585,105301976,tractor,MCAD,35.15430,-90.14263,2017-01-01 00:04:10.000,0,0.0,513931.8,1,2,10375468,1,43.2,108.00,12649102
4,12649103,13640272,2017-01-01 00:10:13.187,1564,105351785,tractor,ULAL,39.90268,-84.19254,2017-01-01 00:10:11.000,3,91.0,498691.0,1,2,10375469,1,50.3,125.75,12649103
5,12649104,13640273,2017-01-01 00:11:08.123,1711,105321298,tractor,FOSJA,30.51558,-81.63283,2017-01-01 00:11:10.000,3,127.6,342948.0,1,2,10375470,1,81.7,163.40,12649104
6,12649105,13640274,2017-01-01 00:11:48.803,1693,105356123,tractor,THOCHA,36.18582,-86.26805,2017-01-01 00:11:50.000,0,333.2,340043.7,1,2,10375471,1,37.4,74.80,12649105
7,12649106,13640275,2017-01-01 00:12:26.077,1787,105455670,tractor,RODMAR,35.16910,-80.88830,2017-01-01 00:12:27.000,53,193.1,224306.6,1,2,10375472,1,62.5,125.00,12649106
8,12649107,13640276,2017-01-01 00:13:02.450,1992,105369677,tractor,BATK,30.34646,-81.70727,2017-01-01 00:13:00.000,0,0.0,39895.5,1,0,10375473,1,52.0,104.00,12649107
9,12649108,13640277,2017-01-01 00:13:38.137,1711,105321298,tractor,FOSJA,30.51642,-81.63271,2017-01-01 00:13:40.000,0,96.3,342948.1,2,1,10375474,1,79.8,159.60,12649108


In [80]:
#Fuel_Transactions.query("TankLevelPercent > 100.0")
Fuel_Transactions['EventTimeStamp'].max()

'2018-03-23 08:44:58.640'

In [258]:
C['DriverID'].value_counts().head(1)
#A = pd.DataFrame(C.['Id_y'].value_counts().max())
#A.sort_values(by=['Id_y'], ascending=False)


DYET    33866
Name: DriverID, dtype: int64

In [66]:
##us = gpd.read_file('data\USA_adm1.shp')

In [67]:
##fig, ax = plt.subplots(figsize = (300, 300), subplot_kw = {'aspect': 'equal'})
##us.plot(ax=ax, color="grey")